In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
# school_data_complete = school_data_complete.sort_values(by="school_name")
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
# total number of schools
total_schools = school_data['school_name'].count()

# total number of students
total_students = student_data['student_name'].count()
# student total will need to be formatted - str can't have calculations performed on them
total_students_formatted = "{:,}".format(total_students)

# total budget - help from kite.com
total_budget = "${:,.2f}".format(school_data['budget'].sum())

# average math score
average_math_score = round(school_data_complete['math_score'].mean(), 6)

# average reading score
average_reading_score = round(school_data_complete['reading_score'].mean(), 6)

# percentage of students with passing math score (70+)
plus70_math = 0
for score in school_data_complete['math_score']:
    if score >= 70:
        plus70_math = plus70_math + 1

percent_math_passing = round(((plus70_math / total_students) * 100), 6)

# percentage of students with passing reading score (70+)
plus70_reading = 0
for score in school_data_complete['reading_score']:
    if score >= 70:
        plus70_reading = plus70_reading + 1

percent_reading_passing = round(((plus70_reading / total_students) * 100), 6)

# percentage of students who passed math *and* reading (% Overall Passing)
plus70_overall = 0
for student in range(len(school_data_complete)):
    if school_data_complete['math_score'][student] >= 70 and school_data_complete['reading_score'][student] >= 70:
        plus70_overall = plus70_overall + 1

percent_overall_passing = round(((plus70_overall / total_students) * 100), 6)
print(percent_overall_passing)

65.172326


In [4]:
# dataframe for district summary
# help from Stack Overflow
district_summary = pd.DataFrame({
    "Total Schools" : total_schools,
    "Total Students" : total_students_formatted,
    "Total Budget" : total_budget,
    "Average Math Score" : average_math_score,
    "Average Reading Score" : average_reading_score,
    "% Passing Math" : percent_math_passing,
    "% Passing Reading" : percent_reading_passing,
    "% Passing Overall" : percent_overall_passing}, index=[0])
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [5]:
# initiate dataframe for school summary
# school name, school type, total students, total budget can be pulled from school_data df
school_summary = school_data.loc[:, ["school_name", "type", "size", "budget"]]

# rename columns
school_summary = school_summary.rename(columns={
    "school_name": "School Name", 
    "type": "School Type", 
    "size": "Total Students", 
    "budget": "Total School Budget"})

# reset index
school_summary = school_summary.set_index("School Name")

# sort schools alphabetically
school_summary = school_summary.sort_values(by="School Name")

# instantiate remaining columns of summary
# because we want 'Per Student Budget' to be formatted as currency, we need it to contain str objects
# the rest can hold floats
school_summary['Per Student Budget'] = "0"
school_summary[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']] = 0
school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,3124928,0,0,0,0,0,0
Cabrera High School,Charter,1858,1081356,0,0,0,0,0,0
Figueroa High School,District,2949,1884411,0,0,0,0,0,0
Ford High School,District,2739,1763916,0,0,0,0,0,0
Griffin High School,Charter,1468,917500,0,0,0,0,0,0


In [9]:
# Per Student Budget -----------------------------------------------------
for i in range(len(school_summary)):
    perStudent_budget = school_summary['Total School Budget'][i] / school_summary['Total Students'][i]
    school_summary['Per Student Budget'][i] = "${:,.2f}".format(perStudent_budget)

# # Average Math and Reading Scores by school ------------------------------
# # variables and initial values
totalMath_score = 0
totalReading_score = 0
averageMath_score = 0
averageReading_score = 0
plus70_math = 0
plus70_reading = 0
plus70_overall = 0

# iterate through school_data_complete
# help from Stack Overflow (https://stackoverflow.com/questions/49710722/how-to-implement-where-clause-in-python)
for i in range(len(school_data_complete) - 1):
    # if school name matches name in next row
    if school_data_complete['school_name'][i] == school_data_complete['school_name'][i + 1]:
        # add to math and reading score totals
        totalMath_score = totalMath_score + school_data_complete['math_score'][i]
        totalReading_score = totalReading_score + school_data_complete['reading_score'][i]
        # add to variables containing number of passing students
        if school_data_complete['math_score'][i] >= 70:
            plus70_math = plus70_math + 1
            
        if school_data_complete['reading_score'][i] >= 70:
            plus70_reading = plus70_reading + 1
        
        if school_data_complete['math_score'][i] >= 70 and school_data_complete['reading_score'][i] >= 70:
            plus70_overall = plus70_overall + 1
    
    # if school name doesn't match
    elif school_data_complete['school_name'][i] != school_data_complete['school_name'][i + 1]:
        # final math and reading score totals
        totalMath_score = totalMath_score + school_data_complete['math_score'][i]
        totalReading_score = totalReading_score + school_data_complete['reading_score'][i]
        # final totals of passing students
        if school_data_complete['math_score'][i] >= 70:
            plus70_math = plus70_math + 1
            
        if school_data_complete['reading_score'][i] >= 70:
            plus70_reading = plus70_reading + 1
        
        if school_data_complete['math_score'][i] >= 70 and school_data_complete['reading_score'][i] >= 70:
            plus70_overall = plus70_overall + 1
            
        # calulate average math and reading scores
        averageMath_score = totalMath_score / school_summary.loc[school_data_complete['school_name'][i], "Total Students"]
        averageReading_score = totalReading_score / school_summary.loc[school_data_complete['school_name'][i], "Total Students"]
        # store calculations in respective columns
        school_summary.loc[school_data_complete['school_name'][i], "Average Math Score"] = averageMath_score
        school_summary.loc[school_data_complete['school_name'][i], "Average Reading Score"] = averageReading_score
    
        school_summary.loc[school_data_complete['school_name'][i], "% Passing Math"] = round(((plus70_math / (school_summary.loc[school_data_complete['school_name'][i], "Total Students"])) * 100), 6)
        school_summary.loc[school_data_complete['school_name'][i], "% Passing Reading"] = round(((plus70_reading / (school_summary.loc[school_data_complete['school_name'][i], "Total Students"])) * 100), 6)
        school_summary.loc[school_data_complete['school_name'][i], "% Overall Passing"] = round(((plus70_overall / (school_summary.loc[school_data_complete['school_name'][i], "Total Students"])) * 100), 6)

        # reset variables for next school name
        totalMath_score = 0
        totalReading_score = 0
        averageMath_score = 0
        averageReading_score = 0
        plus70_math = 0
        plus70_reading = 0
        plus70_overall = 0

# school_summary.head()
print(school_summary.loc["Thomas High School", :])

/var/folders/6j/g7z_h15n2xb6lf5plnw73lrm0000gn/T/ipykernel_27067/1399363667.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_summary['Per Student Budget'][i] = "${:,.2f}".format(perStudent_budget)


School Type              Charter
Total Students              1635
Total School Budget      1043130
Per Student Budget       $638.00
Average Math Score           0.0
Average Reading Score        0.0
% Passing Math               0.0
% Passing Reading            0.0
% Overall Passing            0.0
Name: Thomas High School, dtype: object


In [14]:
# math/reading scores and passing percentages for Thomas High School
# variables and initial values
totalMath_Thomas = 0
totalReading_Thomas = 0
averageMath_Thomas = 0
averageReading_Thomas = 0
plus70Thomas_math = 0
plus70Thomas_reading = 0
plus70Thomas_overall = 0

# iterate through school_data_complete
for i in range(len(school_data_complete)):
    # pull scores for Thomas High School
    if school_data_complete['school_name'][i] == "Thomas High School":
        # add to math and reading score totals
        totalMath_Thomas = totalMath_Thomas + school_data_complete['math_score'][i]
        totalReading_Thomas = totalReading_Thomas + school_data_complete['reading_score'][i]
        # add to variables containing number of passing students
        if school_data_complete['math_score'][i] >= 70:
            plus70Thomas_math = plus70_math + 1
            
        if school_data_complete['reading_score'][i] >= 70:
            plus70Thomas_reading = plus70_reading + 1
        
        if school_data_complete['math_score'][i] >= 70 and school_data_complete['reading_score'][i] >= 70:
            plus70Thomas_overall = plus70_overall + 1
            
        # calulate average math and reading scores
        averageMath_Thomas = totalMath_Thomas / school_summary.loc["Thomas High School", "Total Students"]
        averageReading_Thomas = totalReading_Thomas / school_summary.loc["Thomas High School", "Total Students"]
        # store calculations in respective columns
        school_summary.loc["Thomas High School", "Average Math Score"] = averageMath_Thomas
        school_summary.loc["Thomas High School", "Average Reading Score"] = averageReading_Thomas
    
        school_summary.loc["Thomas High School", "% Passing Math"] = round(((plus70Thomas_math / (school_summary.loc["Thomas High School", "Total Students"])) * 100), 6)
        school_summary.loc["Thomas High School", "% Passing Reading"] = round(((plus70Thomas_reading / (school_summary.loc["Thomas High School", "Total Students"])) * 100), 6)
        school_summary.loc["Thomas High School", "% Overall Passing"] = round(((plus70Thomas_overall / (school_summary.loc["Thomas High School", "Total Students"])) * 100), 6)

print(school_summary.loc["Thomas High School", :])

School Type                Charter
Total Students                1635
Total School Budget        1043130
Per Student Budget         $638.00
Average Math Score       83.418349
Average Reading Score     83.84893
% Passing Math           93.333333
% Passing Reading        97.370031
% Overall Passing        91.009174
Name: Thomas High School, dtype: object


In [15]:
# Total School Budget as currency - help from Stack Overflow (https://stackoverflow.com/questions/35019156/pandas-format-column-as-currency)
school_summary['Total School Budget'] = school_summary['Total School Budget'].apply(lambda x: "${:,.2f}".format(x))
school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455


In [16]:
# top performing schools - sort and display the top 5 schools by % overall passing
topPerforming_summary = school_summary.sort_values(by='% Overall Passing', ascending=False)
topPerforming_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.333333,97.370031,91.009174
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [17]:
# bottom performing schools - sort and display the bottom 5 schools by % overall passing
bottomPerforming_summary = school_summary.sort_values(by='% Overall Passing')
bottomPerforming_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [ ]:
# average math scores by grade


In [ ]:
# average reading scores by grade

In [ ]:
# scores by school spending
# Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)

In [ ]:
# scores by school size
# Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)

In [ ]:
# scores by school type
# Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)